# Experimenting with Cleaning, Clustering & Summarization Pipelines

### To do (technical)
- Implement date windows on my corpus loader function

In [1]:
import os
import re
import json

import numpy as np
import pandas as pd
import networkx as nx

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

In [2]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "/home/ozwald/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "disaster"

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

### First, get a list of all the news dumps created so far

In [3]:
corpus = helper.load_clean_corpus(storage_path, corpus_type)

Total files: 77
Loading file: bing_disaster_corpus_2019-12-17_0022.json
Loading file: bing_disaster_corpus_2020-01-11_0022.json
Loading file: bing_disaster_corpus_2019-12-27_1803.json
Loading file: bing_disaster_corpus_2019-12-19_0022.json
Loading file: bing_disaster_corpus_2019-12-08_0023.json
Loading file: bing_disaster_corpus_2019-11-17_1952.json
Loading file: bing_disaster_corpus_2020-01-20_1222.json
Loading file: bing_disaster_corpus_2019-11-25_0022.json
Loading file: bing_disaster_corpus_2020-01-18_1658.json
Loading file: bing_disaster_corpus_2019-12-01_1222.json
Loading file: bing_disaster_corpus_2019-12-06_1222.json
Loading file: bing_disaster_corpus_2019-12-12_0022.json
Loading file: bing_disaster_corpus_2019-11-19_0022.json
Loading file: bing_disaster_corpus_2019-11-17_1956.json
Loading file: bing_disaster_corpus_2020-01-09_1222.json
Loading file: bing_disaster_corpus_2019-12-18_1222.json
Loading file: bing_disaster_corpus_2020-01-08_0022.json
Loading file: bing_disaster_corp

In [4]:
corpus.head()

,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
0,Usman vs Covington live stream: Free links to ...,Colby Covington is set to take on Kamaru Usman...,2019-12-17T00:02:00.0000000Z,https://www.independent.co.uk/sport/general/mm...,www.independent.co.uk,2019-12-17 00:21:31.786528,bing_news_api,Usman vs Covington live stream: Free links to ...
6,<b>Flood</b> warnings in place across Berkshir...,A number of <b>flood</b> warnings are in place...,2019-12-16T13:26:00.0000000Z,https://www.getreading.co.uk/news/reading-berk...,www.getreading.co.uk,2019-12-17 00:21:31.787132,bing_news_api,Flood warnings in place across Berkshire after...
11,<b>Flood</b> alert issued for Burton after rai...,People across Burton and South Derbyshire are ...,2019-12-16T09:17:00.0000000Z,https://www.derbytelegraph.co.uk/burton/flood-...,www.derbytelegraph.co.uk,2019-12-17 00:21:31.787399,bing_news_api,Flood alert issued for Burton after rainfall. ...
13,Swindon <b>flood</b> defence pond overflows an...,Pavements and a park have been left flooded du...,2019-12-16T17:57:00.0000000Z,https://www.bbc.co.uk/news/uk-england-wiltshir...,www.bbc.co.uk,2019-12-17 00:21:31.787503,bing_news_api,Swindon flood defence pond overflows and cause...
14,<b>Flood</b> Warnings issued for River Severn ...,Heavy rain has seen the Environment Agency put...,2019-12-16T11:57:00.0000000Z,https://www.gloucestershirelive.co.uk/news/glo...,www.gloucestershirelive.co.uk,2019-12-17 00:21:31.787551,bing_news_api,Flood Warnings issued for River Severn as Envi...


In [5]:
corpus = corpus.iloc[-5000:,]

In [6]:
corpus.shape

(4454, 8)

## 2. Clustering using Entity Detection And Network Analytics

This doesn't resolve very well for Bing, because there's a whole bunch of keywords from the original searches in there.  Suspect that's got a lot to do with the failure of the other methods too.  For the network analytics method I'm going to try removing the keywords from the table first.

In [7]:
#with open("/home/ozwald/Dropbox/news_crow/scrape_settings.json", "r") as f:
#    scrape_config = json.load(f)
#
#search_terms = scrape_config['disaster_search_list']
#search_terms = re.sub(r"[^0-9A-Za-z ]", "", " ".join(search_terms)).lower().split()
#search_terms = set(search_terms)

In [8]:
#search_terms

In [9]:
model = reps.NounAdjacencyModel(list(corpus['clean_text']), list(corpus['clean_text']))
model.noun_sets[3]

PARALLEL AWESOMENESS!!!
Is this where ram useage goes off the charts?
found all nouns
Reduced noun lists to sets
Created entity table
Aggregated result table


{'Council',
 'Fields',
 'Margaret',
 'Merton',
 'Parish',
 'St',
 'Stratton',
 'Swindon'}

In [10]:
nouns_df = model.table.copy()
nouns_df.head()

,Water,Korea,Langer,Winston,research,Bangladesh,Haiti,Venice‘s,Bowen,NBA,...,Croft,Shirley,Raze,Elton_John,Ulster,Fairy,quot;lows,1C,Alec,ITF
"Usman vs Covington live stream: Free links to watch UFC 245 flood online as piracy hits ;peak levels;. Colby Covington is set to take on Kamaru Usman at the main event of UFC 245, with the Welterweight title on the line. Unusually, UFC has also put two other title fights on a single pay-per-view in the US, as Max Holloway faces Alexander Volkanovski and Amanda Nunes squares off against Germaine de Randamie. Fight fans will be able to watch the ...",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Flood warnings in place across Berkshire after heavy downpours. A number of flood warnings are in place across Berkshire after heavy downpours at the weekend. People have been warned flooding is possible near many of the county;s rivers and those living or working near by have been urged to be prepared. The flood information service on gov.uk has issued a number of flood alerts and experts are monitoring ...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Flood alert issued for Burton after rainfall. People across Burton and South Derbyshire are being warned to quot;be preparedquot; as a flood alert has been issued across the area. Persistent rainfall has fallen across the last seven days, leading to the warning on the Government website, gov.uk. The flood alert for Burton was issued on the last night, Sunday, December 15, and is still in place today.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Swindon flood defence pond overflows and causes flooding. Pavements and a park have been left flooded during work to install new drainage. The work, at Merton Fields in Swindon, was for an attenuation pond to divert water during heavy rainfall. However, it immediately overflowed in heavy rain and residents reported water pouring under fences and into gardens. Stratton St Margaret Parish Council ...",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Flood Warnings issued for River Severn as Environment Agency says immediate action required. Heavy rain has seen the Environment Agency put Flood Warnings on stretches of the River Severn in Gloucestershire. It says river levels are expected to remain high until Wednesday and flooding is expected and immediate action required. The Flood Warnings are in place for the River Severn at Apperley and The Leigh and on the River Severn at ...,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Drop any noun/noun phrase containing one of the search terms, then create an adjacency matrix

### Drop any noun/phrase occuring too infrequently

In [ ]:
# Get 500 most common nouns
nouns_to_keep = list(nouns_df.\
                    sum(axis=0).\
                    sort_values(ascending=False).\
                    index)

# Cut out any nouns containing the original search terms
#nouns_to_keep = [noun for noun in nouns_to_keep if sum([term in noun for term in search_terms]) == 0]

# Keep only top 500 most common
nouns_to_keep = nouns_to_keep[:500]

# Subset nouns dataframe
nouns_df = nouns_df[nouns_to_keep]

print(nouns_df.shape)

In [ ]:
embeddings = np.asarray(nouns_df)
adjacency = np.dot(embeddings, embeddings.T)
print(np.max(adjacency))

In [ ]:
# If the "lower" limit is 1, the graph has so many edges it eats ALL the memory of my desktop, even
# with just 500-ish stories to process.
upper = 100
lower = 3
G = nx.Graph()
rows, cols = np.where((upper >= adjacency) & (adjacency >= lower))
weights = [float(adjacency[rows[i], cols[i]]) for i in range(len(rows))]
edges = zip(rows.tolist(), cols.tolist(), weights)
G.add_weighted_edges_from(edges)

# Simplify; remove self-edges
G.remove_edges_from(nx.selfloop_edges(G))

In [ ]:
G.number_of_edges()

In [ ]:
#G_plot = nx.petersen_graph()
#plt.subplot(121)
#nx.draw(G, with_labels=True, font_weight='bold')
#plt.subplot(122)
#nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

### Cliques, worth a look?
Idea from the docs, explanation at https://en.wikipedia.org/wiki/Clique_(graph_theory)

So, cliques are allowed to overlap - should've thought of that.  Still, good preliminary results and I've found I can disambiguate the cliques to some degree by cutting out weaker links (fewer shared entities).

I should add it also appears to merely suffer from the same problems as the other clustering methods, clusters are ultimately hierarchical!

In [ ]:
cliques = []
for x in nx.find_cliques(G):
    x.sort()
    cliques.append((len(x), x))

In [ ]:
cliques_df = pd.DataFrame({"nodes_list": [x[1] for x in cliques],
                           "clique_size": [x[0] for x in cliques]}).\
                    sort_values("clique_size", ascending=False).\
                    reset_index()

In [ ]:
len(cliques_df[cliques_df['clique_size'] >= 5])

In [ ]:
cliques_df[cliques_df['clique_size'] >= 5]

In [ ]:
# Useful flatten function from Alex Martelli on https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
cliqued = set(flatten(list(cliques_df['nodes_list'])))
len(cliqued)

In [ ]:
for node in cliques_df.iloc[0]['nodes_list']:
    print(corpus.iloc[node]['clean_text'])

In [ ]:
for node in cliques_df.iloc[1]['nodes_list']:
    print(corpus.iloc[node]['clean_text'])

In [ ]:
for node in cliques_df.iloc[3]['nodes_list']:
    print(corpus.iloc[node]['clean_text'])

In [ ]:
for node in cliques_df.iloc[17]['nodes_list']:
    print(corpus.iloc[node]['clean_text'])

### Connected components

In [ ]:
nx.number_connected_components(G)

In [ ]:
components = [component for component in nx.connected_components(G)]

In [ ]:
sum([len(component) for component in components])

### Community Detection Algorithm

In [ ]:
from community import best_partition

In [ ]:
# Apply Louvain Community Detection
# The keys are nodes, the values are the partitions they belong to
partition = best_partition(G)

number_partitions = max(partition.values())
number_partitions

In [ ]:
# Iterate through and get a list of partitions and their nodes
partition_contents = {}
for key in partition.keys():
    partition_contents[partition[key]] = partition_contents.get(partition[key], []) + [key]

# Drop partitions that are too small
for key in list(partition_contents.keys()):
    if len(partition_contents[key]) < 5:
        partition_contents.pop(key)

In [ ]:
# Let's see how big our "clusters" are, and how many there are total after removing the tiny ones
partition_lengths = {key:len(value) for key, value in partition_contents.items()}
print(partition_lengths, sum(partition_lengths))

In [ ]:
for node in partition_contents[2][:10]:
    print(corpus.iloc[node]['clean_text'])

In [ ]:
for node in partition_contents[5][:10]:
    print(corpus.iloc[node]['clean_text'])

In [ ]:
for node in partition_contents[10][:10]:
    print(corpus.iloc[node]['clean_text'])

In [ ]:
for node in partition_contents[14][:10]:
    print(corpus.iloc[node]['clean_text'])